In [1]:
# import necessary stuff and python-wrapper of verse
import os
import pprint
import numpy as np
import json
import sys
import codecs
from scipy.sparse import csr_matrix

from verse.python.wrapper import VERSE
from multi_class_classification import MultiClassClassification
from multi_label_classification import MultiLabelClassification
from clustering import Clustering
from link_prediction import LinkPrediction
from experiment import Experiment

In [2]:
# initialize pretty printer
pp = pprint.PrettyPrinter(indent=4, depth=8)

In [3]:
# define path to c++ inplementation of verse and its *.so compiled files
# instantiate verse algorithm object
cpath = os.path.dirname(os.path.realpath('verse/src/verse.cpp'))
verse = VERSE(cpath=cpath)

In [4]:
# construct example adjacency matrix in csr format
row = np.array([0, 0, 1, 2, 2, 2])
col = np.array([0, 2, 2, 0, 1, 2])
data = np.array([1, 2, 3, 4, 5, 6])

csr_adjacency_matrix = csr_matrix((data, (row, col)), shape=(10, 10))

In [5]:
# define hyper-parameters for verse with personalized page rank as sim_G function
graph = csr_adjacency_matrix
w = None
n_hidden = 128
alpha = 0.85
steps = 100000
n_neg_samples = 3
lr = 0.0025
rng_seed = 0
n_threads = -1
num_of_reps = 2
random_seed = 42
random_seeds = list(range(42,42+num_of_reps)) 

In [6]:
# configure telegram notifier bot
my_telegram_config = {
    "telegram": {
        "token": "350553078:AAEu70JDqMFcG_x5eBD3nqccTvc4aFNMKkg",
        "chat_id": "126551968",
        "verbose": 1
    }
}

In [7]:
# compute node embeddings
# ATTENTION: takes a while until termination and cannot be terminated through restarting kernel
# Need to kill terminal, where jupyter notebook is started in in order to kill c++-process
verse_ppr_embeddings = verse.verse_ppr(graph, w, n_hidden, alpha, steps, n_neg_samples, lr, rng_seed, n_threads)

In [8]:
# or read *.bin file with precomputed embeddings
embeddings_file_path = 'data/test_converter_verse_embeddings.bin'
embeddings_file = open(embeddings_file_path, "r")
embeddings_file_content = np.fromfile(embeddings_file, dtype=np.float32)
num_of_nodes = int(np.shape(embeddings_file_content)[0] / n_hidden)
verse_ppr_embeddings = embeddings_file_content.reshape((num_of_nodes, n_hidden))

In [9]:
np.shape(verse_ppr_embeddings)

(10, 128)

In [10]:
# instantiate multi-class classification expirement
verse_ppr_classification_experiment = MultiClassClassification(method_name='Verse-PPR', dataset_name='Test-Data',
                                                               performance_function='both', train_size=0.5,
                                                               embeddings=verse_ppr_embeddings,
                                                               node_labels=[0,1,1,0,0,1,1,0,0,1])

Initialize multi-class classification experiment with Verse-PPR on Test-Data evaluated through both on 50.0% train data!


In [11]:
# train, predict and evaluate multi-class classification experiment
train_test_split_classification = verse_ppr_classification_experiment.preprocess_data(random_seed=random_seed)
logistic_regression_model = verse_ppr_classification_experiment.train(random_seed=random_seed)
node_label_predictions = verse_ppr_classification_experiment.predict()
verse_ppr_classification_experiment_results = verse_ppr_classification_experiment.evaluate()

Train multi-class classification experiment with Verse-PPR on Test-Data evaluated through both on 50.0% train data!
convergence after 29 epochs took 0 seconds
Trained multi-class classification experiment in 0.11 sec.!
Predict multi-class classification experiment with Verse-PPR on Test-Data evaluated through both on 50.0% train data!
Predicted multi-class classification experiment in 0.0 sec.!
Evaluate multi-class classification experiment with Verse-PPR on Test-Data evaluated through both on 50.0% train data!


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.0s finished


In [12]:
# print results of multi-class classification experiment
verse_ppr_classification_experiment_results

{'macro': 0.4, 'micro': 0.4000000000000001}

In [13]:
# instantiate clustering expirement
verse_ppr_clustering_experiment = Clustering(method_name='Verse-PPR', dataset_name='Test-Data', 
                                             performance_function='both', embeddings=verse_ppr_embeddings,
                                             node_labels=[0,0,1,1,2,2,1,0,0,2], n_clusters=3)

Initialize clustering experiment with Verse-PPR on Test-Data evaluated through both!


In [14]:
# train, predict and evaluate clustering experiment
train_test_split_clustering = verse_ppr_clustering_experiment.preprocess_data(random_seed=random_seed)
k_means = verse_ppr_clustering_experiment.train(random_seed=random_seed)
node_label_predictions = verse_ppr_clustering_experiment.predict()
verse_ppr_clustering_experiment_results = verse_ppr_clustering_experiment.evaluate()

Train clustering experiment with Verse-PPR on Test-Data evaluated through both!
Trained clustering experiment in 0.21 sec.!
Predict clustering experiment with Verse-PPR on Test-Data evaluated through both!
Predicted clustering experiment in 0.0 sec.!
Evaluate clustering experiment with Verse-PPR on Test-Data evaluated through both!


In [15]:
# print results of clustering experiment
verse_ppr_clustering_experiment_results

{'nmi': 0.15419702222078943, 'silhouette': -0.16295036673545837}

In [16]:
# instantiate multi-label classification expirement
verse_ppr_multi_label_classification_experiment = \
    MultiLabelClassification(method_name='Verse-PPR', dataset_name='Test-Data', performance_function='both',
                             embeddings=verse_ppr_embeddings, n_neighbors=3, classifier='logistic_regression',
                             node_labels=[[0],[1,2],[0,2],[0,1],[2],[1],[0,1],[1],[2],[1,0]])

Initialize multi-label classification experiment with Verse-PPR on Test-Data evaluated through both on 30.0% train data!


In [17]:
# train, predict and evaluate multi-label classification experiment
train_test_split_multi_label_classification = verse_ppr_multi_label_classification_experiment.preprocess_data(random_seed=random_seed)
multi_label_model = verse_ppr_multi_label_classification_experiment.train(random_seed=random_seed)
node_label_predictions = verse_ppr_multi_label_classification_experiment.predict()
verse_ppr_multi_label_classification_experiment_results = verse_ppr_multi_label_classification_experiment.evaluate()

Train multi-label classification experiment with Verse-PPR on Test-Data evaluated through both on 30.0% train data!
convergence after 42 epochs took 0 seconds


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.0s finished


convergence after 42 epochs took 0 seconds
convergence after 42 epochs took 0 seconds


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.0s finished


Trained multi-label classification experiment in 0.3 sec.!
Predict multi-label classification experiment with Verse-PPR on Test-Data evaluated through both on 30.0% train data!
Predicted multi-label classification experiment in 0.0 sec.!
Evaluate multi-label classification experiment with Verse-PPR on Test-Data evaluated through both on 30.0% train data!


In [18]:
# print results of multi-label classification experiment
verse_ppr_multi_label_classification_experiment_results

{'macro': 0.2619047619047619, 'micro': 0.28571428571428564}

In [19]:
# instantiate link prediction expirement
verse_ppr_link_prediction_experiment = \
    LinkPrediction(method_name='Verse-PPR', dataset_name='Test-Data', performance_function='both', 
                   node_embeddings=verse_ppr_embeddings, new_edges=[[1,3],[5,6],[3,6],[1,5],[8,7],[9,4],[7,2]],
                   vector_operator='hadamard', neg_edges=[[2,3],[7,6],[1,6],[2,5],[8,9],[9,3],[7,5]],
                  edge_list=[[1,3],[5,6],[3,6],[1,5],[8,7],[9,4],[7,2]])

100%|██████████| 7/7 [00:00<00:00, 13548.74it/s]

Initialize link prediction experiment with Verse-PPR on Test-Data evaluated through both on 50.0% train data!
Calculate ids for nodes
Compute edgewise features based on hadamard operator!


In [20]:
# train, predict and evaluate link prediction experiment
train_test_split_link_prediction = verse_ppr_link_prediction_experiment.preprocess_data(random_seed=random_seed)
link_prediction_model = verse_ppr_link_prediction_experiment.train(random_seed=random_seed)
edge_label_predictions = verse_ppr_link_prediction_experiment.predict()
verse_ppr_link_prediction_experiment_results = verse_ppr_link_prediction_experiment.evaluate()

Train link prediction experiment with Verse-PPR on Test-Data evaluated through both on 50.0% train data!
convergence after 18 epochs took 0 seconds
Trained link prediction experiment in 0.1 sec.!
Predict link prediction experiment with Verse-PPR on Test-Data evaluated through both on 50.0% train data!
Predicted link prediction experiment in 0.0 sec.!
Evaluate link prediction experiment with Verse-PPR on Test-Data evaluated through both on 50.0% train data!


/home/julius/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.0s finished


In [21]:
# print results of link prediction experiment
#verse_ppr_link_prediction_experiment_results

In [22]:
# init clustering experiment on verse-ppr embeddings with cross-product of experiment_params, each 10x repeated 
verse_ppr_experiment = Experiment(method_name='Verse-PPR', dataset_name='Test-Data', performance_function='both',
                                  node_labels=[0,0,1,1,2,2,1,0,0,2],embeddings_file_path=embeddings_file_path,
                                  node_embedings=None, embedding_dimensionality=128, repetitions=num_of_reps,
                                  experiment_params={'n_clusters': [2,3]},
                                  results_file_path='results/test_verse_ppr_experiment_results.json',
                                  random_seeds=random_seeds, experiment_type='clustering',
                                  pickle_path='results/test_verse_ppr_experiment_results.p',
                                  telegram_config=my_telegram_config)

In [23]:
pp.pprint(my_telegram_config)

{   'telegram': {   'chat_id': '126551968',
                    'token': '350553078:AAEu70JDqMFcG_x5eBD3nqccTvc4aFNMKkg',
                    'verbose': 1}}


In [24]:
# run experiment wrapper: train, predict and evaluate clustering with each param combination 10 times
verse_ppr_experiment_results = verse_ppr_experiment.run()

Start clustering experiment on Test-Data data set with Verse-PPR embeddings
Repeated 2 times and evaluated through bothperformance function(s)
Initialize clustering experiment with Verse-PPR on Test-Data evaluated through both!
Train clustering experiment with Verse-PPR on Test-Data evaluated through both!
Train clustering experiment with Verse-PPR on Test-Data evaluated through both!


/home/julius/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/home/julius/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)


Trained clustering experiment in 0.0 sec.!
Predict clustering experiment with Verse-PPR on Test-Data evaluated through both!
Predicted clustering experiment in 0.0 sec.!
Trained clustering experiment in 0.01 sec.!
Predict clustering experiment with Verse-PPR on Test-Data evaluated through both!
Predicted clustering experiment in 0.0 sec.!
Evaluate clustering experiment with Verse-PPR on Test-Data evaluated through both!
Evaluate clustering experiment with Verse-PPR on Test-Data evaluated through both!
Initialize clustering experiment with Verse-PPR on Test-Data evaluated through both!
Train clustering experiment with Verse-PPR on Test-Data evaluated through both!
Train clustering experiment with Verse-PPR on Test-Data evaluated through both!


/home/julius/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/home/julius/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py:547: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)


Trained clustering experiment in 0.01 sec.!
Predict clustering experiment with Verse-PPR on Test-Data evaluated through both!
Predicted clustering experiment in 0.0 sec.!
Evaluate clustering experiment with Verse-PPR on Test-Data evaluated through both!
Trained clustering experiment in 0.01 sec.!
Predict clustering experiment with Verse-PPR on Test-Data evaluated through both!
Predicted clustering experiment in 0.0 sec.!
Evaluate clustering experiment with Verse-PPR on Test-Data evaluated through both!
Finished clustering experiment on Test-Data data set with Verse-PPR embeddings
Saved results in file results/test_verse_ppr_experiment_results.json
Saved experiment as pickle-model in file results/test_verse_ppr_experiment_results.p


In [25]:
pp.pprint(verse_ppr_experiment_results)

{   'dataset': 'Test-Data',
    'embedding_file': 'data/test_converter_verse_embeddings.bin',
    'method': 'Verse-PPR',
    'parameterizations': [   {   'params': {'n_clusters': 2},
                                 'runs': [   {   'evaluation': {   'nmi': 0.0049311472614544695,
                                                                   'silhouette': 0.2900736927986145},
                                                 'experiment': 140608036956816,
                                                 'random_seed': 42,
                                                 'run': 1},
                                             {   'evaluation': {   'nmi': 0.17490772832691745,
                                                                   'silhouette': 0.15746554732322693},
                                                 'experiment': 140608036957040,
                                                 'random_seed': 43,
                                                 'run': 2}]},
  

In [26]:
# init link prediction experiment on verse-ppr embeddings with cross-product of experiment_params, each 10x repeated 
verse_ppr_link_pred_experiment = Experiment(method_name='Verse-PPR', dataset_name='Test-Data',
                                            performance_function='both',node_labels=[0,0,1,1,2,2,1,0,0,2],
                                            embeddings_file_path=embeddings_file_path, node_embedings=None,
                                            embedding_dimensionality=128, repetitions=num_of_reps,
                                            experiment_params={
                                                'edge_list': [[[1,3],[5,6],[3,6],[1,5],[8,7],[9,4],[7,2]]],
                                                'new_edges':[[[1,3],[5,6],[3,6],[1,5],[8,7],[9,4],[7,2]]],
                                                'neg_edges':[[[2,3],[7,6],[1,6],[2,5],[8,9],[9,3],[7,5]]],
                                                'vector_operator': ['hadamard']
                                            },
                                            results_file_path='results/test_verse_ppr_link_pred_experiment_results.json',
                                            random_seeds=random_seeds, experiment_type='link_prediction')

In [27]:
# run experiment wrapper: train, predict and evaluate link prediction with each param combination 10 times
verse_ppr_link_pred_experiment_results = verse_ppr_link_pred_experiment.run()

100%|██████████| 7/7 [00:00<00:00, 11683.30it/s]

Start link_prediction experiment on Test-Data data set with Verse-PPR embeddings
Repeated 2 times and evaluated through bothperformance function(s)
Initialize link prediction experiment with Verse-PPR on Test-Data evaluated through both on 50.0% train data!
Calculate ids for nodes
Compute edgewise features based on hadamard operator!



/home/julius/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
/home/julius/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Train link prediction experiment with Verse-PPR on Test-Data evaluated through both on 50.0% train data!
Train link prediction experiment with Verse-PPR on Test-Data evaluated through both on 50.0% train data!
convergence after 18 epochs took 0 seconds
convergence after 21 epochs took 0 seconds


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.0s finished


Trained link prediction experiment in 0.11 sec.!
Predict link prediction experiment with Verse-PPR on Test-Data evaluated through both on 50.0% train data!
Predicted link prediction experiment in 0.0 sec.!
Evaluate link prediction experiment with Verse-PPR on Test-Data evaluated through both on 50.0% train data!
Trained link prediction experiment in 0.12 sec.!
Predict link prediction experiment with Verse-PPR on Test-Data evaluated through both on 50.0% train data!
Predicted link prediction experiment in 0.0 sec.!
Evaluate link prediction experiment with Verse-PPR on Test-Data evaluated through both on 50.0% train data!
Finished link_prediction experiment on Test-Data data set with Verse-PPR embeddings
Saved results in file results/test_verse_ppr_link_pred_experiment_results.json


In [28]:
pp.pprint(verse_ppr_link_pred_experiment_results)

{   'dataset': 'Test-Data',
    'embedding_file': 'data/test_converter_verse_embeddings.bin',
    'method': 'Verse-PPR',
    'parameterizations': [   {   'params': {   'edge_list': [   [1, 3],
                                                                [5, 6],
                                                                [3, 6],
                                                                [1, 5],
                                                                [8, 7],
                                                                [9, 4],
                                                                [7, 2]],
                                               'neg_edges': [   [2, 3],
                                                                [7, 6],
                                                                [1, 6],
                                                                [2, 5],
                                                                [8, 9],
              